In [1]:
!pip install pandas
!pip install numpy
import pandas as pd
import numpy as np  
import os

!pip install seaborn
import seaborn as sns

from pandas.api.types import CategoricalDtype
Train_label = pd.read_csv('Training set labels.csv')
Train_value = pd.read_csv('Training set values.csv')
Test_value = pd.read_csv('Test set values.csv')
Train = pd.merge(Train_value,Train_label, how='inner',left_on='id',right_on='id', left_index=False)

  Using cached pandas-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (10.8 MB)
  Using cached numpy-1.21.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
  Using cached scipy-1.7.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Using cached matplotlib-3.4.2-cp37-cp37m-manylinux1_x86_64.whl (10.3 MB)
  Using cached kiwisolver-1.3.1-cp37-cp37m-manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Pillow-8.3.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (3.0 MB)


In [2]:
Train.shape

(59400, 41)

In [3]:
Train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [4]:
Train.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [5]:
Train[['longitude','latitude']].describe()

,longitude,latitude
count,59400.000000,5.940000e+04
mean,34.077427,-5.706033e+00
std,6.567432,2.946019e+00
min,0.000000,-1.164944e+01
25%,33.090347,-8.540621e+00
50%,34.908743,-5.021597e+00
75%,37.178387,-3.326156e+00
max,40.345193,-2.000000e-08


In [6]:
# Reference: https://worldpopulationreview.com/country-locations/where-is-tanzania    
# Range of Most Extreme Points in Every Direction of Tanzania
# latitude :  00°59' - 11°45′ -> should be negative due South Lat  -> -0.59  ~ - 11.45
# longitude:  29°10' -  40°29′ East
long_lat_error =Train[((Train['longitude'] < 29.10)  | (Train['longitude'] > 40.29)) | ((Train['latitude'] < -11.45)  | (Train['latitude'] > -0.59))] 
long_lat_error

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
21,6091,0.0,2013-02-10,Dwsp,0,DWE,0.000000,-2.000000e-08,Muungano,0,...,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump,functional
53,32376,0.0,2011-08-01,Government Of Tanzania,0,Government,0.000000,-2.000000e-08,Polisi,0,...,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
168,72678,0.0,2013-01-30,Wvt,0,WVT,0.000000,-2.000000e-08,Wvt Tanzania,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
177,56725,0.0,2013-01-17,Netherlands,0,DWE,0.000000,-2.000000e-08,Kikundi Cha Wakina Mama,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
253,13042,0.0,2012-10-29,Hesawa,0,DWE,0.000000,-2.000000e-08,Kwakisusi,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59208,3631,0.0,2013-01-22,Dwsp,0,DWE,0.000000,-2.000000e-08,Mtakuja,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
59295,60843,0.0,2011-07-19,Rwssp,0,DWE,0.000000,-2.000000e-08,Maendeleo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
59322,44114,6000.0,2013-02-14,Unicef,532,UNICEF,36.985457,-1.146723e+01,Misepelo,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional needs repair
59324,748,0.0,2013-01-22,World Vision,0,World Vision,0.000000,-2.000000e-08,Mwazwilo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [7]:
# Correct long_lat_error using mean long & lat
long_mean = Train[(Train['longitude'] >= 29.10)  | (Train['longitude'] <= 40.29)]['longitude'].mean()
lat_mean = Train[(Train['latitude'] >= -11.45)  | (Train['latitude'] <= -0.59)]['latitude'].mean()
#long_mean: 34.077426692028794
#lat_mean: -5.706032659626431
                  

In [8]:
Train['longitude'] = np.where(((Train.longitude <29.10 ) | (Train.longitude>40.29)),long_mean,Train.longitude)
Train['latitude'] = np.where(((Train.latitude < -11.45) | (Train.latitude > -0.59)),lat_mean,Train.latitude)

In [9]:
# Test long_lat_error -> should be zero instance 
long_lat_error =Train[((Train['longitude'] < 29.10)  | (Train['longitude'] > 40.29)) | ((Train['latitude'] < -11.45)  | (Train['latitude'] > -0.59))] 
long_lat_error # zero instance
Train[['longitude','latitude']].describe()

,longitude,latitude
count,59400.000000,59400.000000
mean,35.114020,-5.871996
std,2.571584,2.758860
min,29.607122,-11.449849
25%,33.354079,-8.524319
50%,34.905440,-5.440285
75%,37.173618,-3.398151
max,40.287471,-0.998464


In [33]:
#region 
Train['region'].value_counts()
len(Train['region'].value_counts())
df = Train['region'].value_counts().to_frame()
df = df[:7]
df.iloc[:,0]

Train.region.__contains__(df.index)
#exists = Train.region in df.index




TypeError: unhashable type: 'Index'

In [15]:
# Def to set up # of category for features
test_cat = 8
def max_cat(col):
    if len(col.value_counts()) > (test_cat-1):
        df = Train['region'].value_counts().to_frame()
        df = df[:(test_cat-1)]
    return col    
        
        

In [30]:
max_cat(Train['region'])

Train['region'].value_counts()
Train.region.columns
#region_code 
#district_code 
#lga 
#ward 
#population 

AttributeError: 'Series' object has no attribute 'columns'